Welcome to Pandas for Data Science
Todays agenda:

    Data Manipulation
    Concatenating Dataframes
    Dealing with Null Values
    Aggregate Statistics (Groupby)
    Regular Expressions and Dataframes
    
    Working with large amounts of data
    Duckdb (mention)

DATA MANIPULATION

In [ ]:
import pandas as pd
import numpy as np

print ('pandas version', pd.__version__)

# Tokyo Olympics
tokyo_data = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/tokyo_olympics.csv')

# index_col takes a column name or integer value. It doesn't take a boolean or None value.
#tokyo_data = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/tokyo_olympics.csv', index_col=0)
#tokyo_data = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/tokyo_olympics.csv', index_col=False)
#tokyo_data = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/tokyo_olympics.csv', index_col=None)

tokyo_data.head()

tokyo_data['firstname'] = tokyo_data['person_name'].str.split(' ').str[0]
tokyo_data

# Convert string object to title case. Firstname is the second element of the new list (index 1)
tokyo_data['firstname'] = tokyo_data['person_name'].str.split(' ').str[1].str.title()

# Convert string object to title case. Firstname is the last element of the new list (index -1)
# It is better to use the name at the end of the list. Some athletes have two last names.
tokyo_data['firstname'] = tokyo_data['person_name'].str.split(' ').str[-1].str.title()
tokyo_data
tokyo_data.query('firstname == "Tamara"')

tokyo_data.head()
tokyo_data.info()


# You can create a new column by performing some mathematical operation on an existing column. For example:
tokyo_data['weight_pounds'] = tokyo_data['weight_kg'] * 2.20462
tokyo_data['weight_pounds'] = round(tokyo_data['weight_pounds'],2)


# Illustration of the axis:
tokyo_data[['weight_kg','weight_pounds']]
tokyo_data[['weight_kg','weight_pounds']].mean(axis=1)  # Axis 1 will act on all the COLUMNS in each ROW # Axis 0 will act on all the ROWS in each COLUMN
tokyo_data[['weight_kg','weight_pounds']].mean(axis=0)  # Axis 0 will act on all the ROWS in each COLUMN # Axis 1 will act on all the COLUMNS in each ROW
tokyo_data[['weight_kg','weight_pounds']]

# Rename a column by passing a dictionary as an argument to the .rename method
tokyo_data.rename(columns={'weight_pounds':'weight_lb'})
tokyo_data.head()

# You can create a new data frame like this:
tokyo_data2 = tokyo_data.rename(columns={'weight_pounds':'weight_lb'})
tokyo_data2.head()

# Add the birth year for each athlete
tokyo_data.info()

tokyo_data['born_date'] = pd.to_datetime(tokyo_data['born_date'])
tokyo_data.head()
tokyo_data.info()

# You can specify the format for the date, especially since the U.S. uses month-day-year as opposed to other formats
tokyo_data['born_date'] = pd.to_datetime(tokyo_data['born_date'], format="%Y-%m-%d")

# .dt is a method for date manipulations, similar to .str for string manipulations
tokyo_data['birth_year'] = tokyo_data['born_date'].dt.year
tokyo_data[['firstname', 'birth_year']]

# Using Lambda functions with data frames
tokyo_data['height_category'] = tokyo_data['height_cm'].apply(lambda x: 'Short' if x < 140 else ('Average-height' if x < 160 else 'Tall'))
tokyo_data.head()


def height_cat(athlete):
  if athlete['height_cm'] < 172 and athlete['weight_kg'] < 68:
    return 'Lightweight'
  if athlete['height_cm'] < 192 and athlete['weight_kg'] < 81:
    return 'Middleweight'
  else:
    return 'Heavyweight'

# Act on all the columns in each row
tokyo_data['height_category'] = tokyo_data.apply(height_cat, axis=1)
tokyo_data.head(29)


# FILTERING DATA
mask = tokyo_data['height_category'] != 'Heavyweight'
tokyo_data[tokyo_data['height_category'] != 'Heavyweight']
tokyo_data[mask]


pandas version 2.1.4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   person_name  20 non-null     object 
 1   country      20 non-null     object 
 2   discipline   20 non-null     object 
 3   born_date    20 non-null     object 
 4   height_cm    20 non-null     float64
 5   weight_kg    20 non-null     float64
 6   firstname    20 non-null     object 
dtypes: float64(2), object(5)
memory usage: 1.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   person_name    20 non-null     object 
 1   country        20 non-null     object 
 2   discipline     20 non-null     object 
 3   born_date      20 non-null     object 
 4   height_cm      20 non-null     float64
 5   weight_kg      20 non-null     f

,person_name,country,discipline,born_date,height_cm,weight_kg,firstname,weight_pounds,birth_year,height_category
2,ABAGNALE Giovanni,Italy,Rowing,1894-09-11,130.17,76.3,Giovanni,168.21,1894,Middleweight
3,ABALDE Alberto,Spain,Basketball,1899-02-12,155.33,63.8,Alberto,140.65,1899,Lightweight
4,ABALDE Tamara,Spain,Basketball,1896-03-12,135.67,75.2,Tamara,165.79,1896,Middleweight
7,ABASS Abobakr,Sudan,Swimming,1886-12-12,155.33,75.0,Abobakr,165.35,1886,Middleweight
8,ABBASALI Hamideh,Islamic Republic,Karate,1882-12-01,135.67,76.3,Hamideh,168.21,1882,Middleweight
10,ABBINGH Lois,Netherlands,Handball,1899-02-12,130.17,75.2,Lois,165.79,1899,Middleweight
14,ABDALLA Maryam,Egypt,Artistic Swimming,1882-12-01,130.17,76.3,Maryam,168.21,1882,Middleweight
15,ABDALLAH Shahd,Egypt,Artistic Swimming,1894-09-11,155.33,63.8,Shahd,140.65,1894,Lightweight
16,ABDALRASOOL Mohamed,Sudan,Judo,1899-02-12,135.67,75.2,Mohamed,165.79,1899,Middleweight


CONCATENATING DATAFRAMES

In [ ]:
import pandas as pd
import numpy as np

print ('pandas version', pd.__version__)

# Tokyo Olympics
tokyo_data = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/tokyo_olympics.csv')

# CONCATENATE DATA FRAMES
# We're interested in the athletes from specific countries;
europe = tokyo_data[tokyo_data['country'].isin(['Spain', 'Netherlands', 'Norway', 'Italy', 'France'])]
africa = tokyo_data[tokyo_data['country'].isin(['Sudan', 'Egypt'])]
# Create a subset for America
# america
europe
africa

# We can also use this logical statement in order to filter by rows:
african_subset = tokyo_data[(tokyo_data['country'] == 'Sudan') | (tokyo_data['country'] == 'Egypt')]
african_subset
# Use a logical operator to create the European subset
# european_subset = tokyo_data[(tokyo_data['country'] == 'A') | (tokyo_data['country'] == 'B')]

# We can create a subset of our dataframe where the isin method evaluates to False
not_african_subset = tokyo_data[tokyo_data['country'].isin(['Sudan', 'Egypt']) == False]
not_african_subset

concat_df = pd.concat([africa,europe])
concat_df


pandas version 2.1.4


,person_name,country,discipline,born_date,height_cm,weight_kg
7,ABASS Abobakr,Sudan,Swimming,1886-12-12,155.33,75.0
14,ABDALLA Maryam,Egypt,Artistic Swimming,1882-12-01,130.17,76.3
15,ABDALLAH Shahd,Egypt,Artistic Swimming,1894-09-11,155.33,63.8
16,ABDALRASOOL Mohamed,Sudan,Judo,1899-02-12,135.67,75.2
17,ABDEL LATIF Radwa,Egypt,Shooting,1896-03-12,195.62,88.3
18,ABDEL RAZEK Samy,Egypt,Shooting,1881-01-12,130.17,99.2
19,ABDELAZIZ Abdalla,Egypt,Karate,1880-12-12,155.33,100.5
0,AALERUD Katrine,Norway,Cycling Road,1886-07-11,135.67,88.0
1,ABAD Nestor,Spain,Artistic Gymnastics,1882-12-01,195.62,75.0
2,ABAGNALE Giovanni,Italy,Rowing,1894-09-11,130.17,76.3


DEALING WITH NULL VALUES

In [ ]:
import pandas as pd
import numpy as np

print ('pandas version', pd.__version__)

# Tokyo Olympics
tokyo_data = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/tokyo_olympics.csv')

# DEALING WITH NULL VALUES
# Assign null values to specific fields
tokyo_data.head()
tokyo_data.loc[[2,3], 'height_cm'] = np.nan
tokyo_data.head()
tokyo_data.info()
# View the number of Na values
tokyo_data.isna().sum()

# You can use .fillna method to fill out the NaN with a different value
tokyo_data.fillna(tokyo_data['height_cm'].mean())

tokyo_data.loc[[2,3], 'height_cm'] = np.nan
tokyo_data['height_cm'] = tokyo_data['height_cm'].interpolate()
tokyo_data.head()

tokyo_data.loc[[2,3], 'height_cm'] = np.nan
tokyo_data.head()
# We can drop the entire row if there is a NaN
tokyo_data.dropna()

# We can drop a row if a subset has NaN
tokyo_data.dropna()
tokyo_data.loc[[2,3], 'height_cm'] = np.nan
tokyo_data.loc[1, 'weight_kg'] = np.nan
tokyo_data.head()
# .dropna does not overwrite the existing data frame
tokyo_data.dropna(subset=['weight_kg'])
tokyo_data.head()
# Assign the resulting data frame to a new variable
tokyo_nonan = tokyo_data.dropna(subset=['weight_kg'])
tokyo_nonan.head()
# You can also update it in memory
tokyo_data = tokyo_data.dropna(subset=['weight_kg'], inplace=True)
tokyo_data.head()

# We can get those rows that have NaN in them:

tokyo_data.loc[[2,3], 'height_cm'] = np.nan
tokyo_data[tokyo_data['height_cm'].isna()]
tokyo_data.head()

# We can get those rows that don't have NaN in them:
tokyo_data.loc[[2,3], 'height_cm'] = np.nan
tokyo_data.head()
tokyo_data[tokyo_data['height_cm'].notna()]



pandas version 2.1.4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   person_name  20 non-null     object 
 1   country      20 non-null     object 
 2   discipline   20 non-null     object 
 3   born_date    20 non-null     object 
 4   height_cm    18 non-null     float64
 5   weight_kg    20 non-null     float64
dtypes: float64(2), object(4)
memory usage: 1.1+ KB


,person_name,country,discipline,born_date,height_cm,weight_kg
0,AALERUD Katrine,Norway,Cycling Road,1886-07-11,135.67,88.0
1,ABAD Nestor,Spain,Artistic Gymnastics,1882-12-01,195.62,NaN
2,ABAGNALE Giovanni,Italy,Rowing,1894-09-11,NaN,76.3
3,ABALDE Alberto,Spain,Basketball,1899-02-12,NaN,63.8
4,ABALDE Tamara,Spain,Basketball,1896-03-12,135.67,75.2


AGGREGATING DATAFRAMES

In [22]:
import pandas as pd
import numpy as np

print ('pandas version', pd.__version__)

# Tokyo Olympics
tokyo_data = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/tokyo_olympics.csv')

# AGGREGATING DATA
tokyo_data['country'].value_counts()

# Get stats generally for athletes from Egypt
tokyo_data[tokyo_data['country'] == 'Egypt'].value_counts()
# Use Logical Operators to combine the conditions:
tokyo_data[(tokyo_data['country'] == 'Egypt') & (tokyo_data['discipline'] == 'Artistic Swimming')].value_counts()
tokyo_data[tokyo_data['country'] == 'Egypt'].value_counts().head(2)
tokyo_data[tokyo_data['country'] == 'Egypt'].value_counts().tail(1)

# Get stats on the discipline for Egyptian athletes

tokyo_data[tokyo_data['country'] == 'Egypt']['discipline'].value_counts()

# A useful Pandas function is .groupby
# We can find the total weight_kg for Artistic Swimmers
tokyo_data.groupby(['discipline'])['weight_kg'].sum()

# Find the average weight of athletes
tokyo_data.groupby(['discipline'])['weight_kg'].mean()

tokyo_data.head()
# agg() method allows you to apply a function or a list of function names
# to be executed along one of the axis of the DataFrame
tokyo_data.groupby(['discipline']).agg({'weight_kg':'sum', 'height_cm':'mean'})

# Group by multiple things
tokyo_data.groupby(['discipline', 'country']).agg({'weight_kg':'sum', 'height_cm':'mean'})

# Convert born_date to datetime and count athletes in each born
tokyo_data['born_date'] = pd.to_datetime(tokyo_data['born_date'])
tokyo_data.info()
tokyo_data.groupby(tokyo_data['born_date'].dt.year).count()
tokyo_data.groupby(tokyo_data['born_date'].dt.year)['discipline'].count().reset_index()
# Sort values with younger athletes at the top of the table and older athletes at the bottom
tokyo_data.groupby(tokyo_data['born_date'].dt.year)['discipline'].count().reset_index().sort_values('born_date', ascending=False)

# Sort values by discipline
tokyo_data.groupby(tokyo_data['born_date'].dt.year)['discipline'].count().reset_index().sort_values('discipline', ascending=False)

tokyo_data['yearborn'] = tokyo_data['born_date'].dt.year
tokyo_data['monthborn'] = tokyo_data['born_date'].dt.month
tokyo_data['monthborn']
tokyo_data['yearborn']
# Sort by the month that the athlete was born
tokyo_data.groupby([tokyo_data['yearborn'], tokyo_data['monthborn']])['discipline'].count().reset_index().sort_values('monthborn', ascending=True)

,person_name,country,discipline,born_date,height_cm,weight_kg,height_rank
0,AALERUD Katrine,Norway,Cycling Road,1886-07-11,135.67,88.0,8.0
1,ABAD Nestor,Spain,Artistic Gymnastics,1882-12-01,195.62,75.0,18.0
2,ABAGNALE Giovanni,Italy,Rowing,1894-09-11,130.17,76.3,3.0
3,ABALDE Alberto,Spain,Basketball,1899-02-12,155.33,63.8,13.0
4,ABALDE Tamara,Spain,Basketball,1896-03-12,135.67,75.2,8.0


REGULAR EXPRESSIONS AND CONDITIONAL CHANGES

In [ ]:
import pandas as pd

print ('pandas version', pd.__version__)

# Use this data, GDP for countries in Europe
data = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/gapminder_gdp_europe.csv')

# Colab
store = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/cal_housing_small.csv')

# Ctrl + Shift + Enter = Specific Line Run
# Ctrl + Enter = All Lines Run

# REGULAR EXPRESSIONS
# Return rows where the country contains the word 'ria' or 'land' # using regular expressions
data[data["country"].str.contains("ria|land")]

# Return rows where the country name starts with a vowel # using regular expressions
data[data["country"].str.contains(r"^[AEIOUaeiou]", na=False)]

# Return rows where the country name ends with a vowel # using regular expressions
data[data["country"].str.contains(r"[AEIOUaeiou]$", na=False)]

# Return rows where the country name contains exactly two vowels # using regular expressions
data[data["country"].str.contains(r"^[^AEIOUaeiou]*[AEIOUaeiou][^AEIOUaeiou]*[AEIOUaeiou][^AEIOUaeiou]*$", na=False)]

# Return rows where the country name contains repeated letters # using regular expressions
data[data["country"].str.contains(r"(.)\1", na=False)]

# Return rows where the country name contains Sp letters at the beginning # using regular expressions
data[data["country"].str.contains(r"(^Sp)", na=False)]

# Return rows where the country name contains ny at the end # using regular expressions
data[data["country"].str.contains(r"(ny$)", na=False)]

# Return rows where the country name contains ria or any at the end # using regular expressions
data[data["country"].str.contains(r"any$|ria$", case=False, na=False)]

# Return rows where the country name contains 8 letters # using regular expressions
data[data["country"].str.contains(r"(^.{8}$)", na=False)]

# How do you return rows where the country name contains 10 letters? # using regular expressions
data[data["country"].str.contains(r"(^.{10}$)", na=False)]

# Return rows where the country name contains 3 or more vowels # using regular expressions
data[data["country"].str.contains(r"([AEIOUaeiou].*){3,}", na=False)]

# Return rows where the country name contains 4 or more vowels # using regular expressions
data[data["country"].str.contains(r"([AEIOUaeiou].*){4,}", na=False)]

# Return rows where the country name contains spaces # using regular expressions
data[data["country"].str.contains(r"[ ]", na=False)]

# Return rows where the country name starts and ends with the same letter # using regular expressions
data[data["country"].str.contains(r"^(.).*\1$", na=False)]

# To turn off regular expressions. Country name contains 3 or more vowels # using regular expressions
data[data["country"].str.contains(r"([AEIOUaeiou].*){3,}", na=False, regex=False)]

# To turn on regular expressions. Country name contains 3 or more vowels # using regular expressions
threemore = data[data["country"].str.contains(r"([AEIOUaeiou].*){3,}", na=False, regex=True)]
threemore
threemore.to_csv('threemoreresult.csv')
newdata = pd.read_csv('threemoreresult.csv')
newdata

pandas version 2.1.4


<ipython-input-7-2f0270f762c0>:28: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[data["country"].str.contains(r"(.)\1", na=False)]
<ipython-input-7-2f0270f762c0>:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[data["country"].str.contains(r"(^Sp)", na=False)]
<ipython-input-7-2f0270f762c0>:34: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[data["country"].str.contains(r"(ny$)", na=False)]
<ipython-input-7-2f0270f762c0>:40: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data[data["country"].str.contains(r"(^.{8}$)", na=False)]
<ipython-input-7-2f0270f762c0>:43: UserWarning: This pattern is interpreted as a regular expressi

,Unnamed: 0,country,gdpPercap_1952,gdpPercap_1957,gdpPercap_1962,gdpPercap_1967,gdpPercap_1972,gdpPercap_1977,gdpPercap_1982,gdpPercap_1987,gdpPercap_1992,gdpPercap_1997,gdpPercap_2002,gdpPercap_2007
0,0,Albania,1601.056136,1942.284244,2312.888958,2760.196931,3313.422188,3533.003910,3630.880722,3738.932735,2497.437901,3193.054604,4604.211737,5937.029526
1,1,Austria,6137.076492,8842.598030,10750.721110,12834.602400,16661.625600,19749.422300,21597.083620,23687.826070,27042.018680,29095.920660,32417.607690,36126.492700
2,2,Belgium,8343.105127,9714.960623,10991.206760,13149.041190,16672.143560,19117.974480,20979.845890,22525.563080,25575.570690,27561.196630,30485.883750,33692.605080
3,3,Bosnia and Herzegovina,973.533195,1353.989176,1709.683679,2172.352423,2860.169750,3528.481305,4126.613157,4314.114757,2546.781445,4766.355904,6018.975239,7446.298803
4,4,Bulgaria,2444.286648,3008.670727,4254.337839,5577.002800,6597.494398,7612.240438,8224.191647,8239.854824,6302.623438,5970.388760,7696.777725,10680.792820
5,5,Croatia,3119.236520,4338.231617,5477.890018,6960.297861,9164.090127,11305.385170,13221.821840,13822.583940,8447.794873,9875.604515,11628.388950,14619.222720
6,6,Czech Republic,6876.140250,8256.343918,10136.867130,11399.444890,13108.453600,14800.160620,15377.228550,16310.443400,14297.021220,16048.514240,17596.210220,22833.308510
7,11,Greece,3530.690067,4916.299889,6017.190733,8513.097016,12724.829570,14195.524280,15268.420890,16120.528390,17541.496340,18747.698140,22514.254800,27538.411880
8,13,Iceland,7267.688428,9244.001412,10350.159060,13319.895680,15798.063620,19654.962470,23269.607500,26923.206280,25144.392010,28061.099660,31163.201960,36180.789190
9,14,Ireland,5210.280328,5599.077872,6631.597314,7655.568963,9530.772896,11150.981130,12618.321410,13872.866520,17558.815550,24521.947130,34077.049390,40675.996350


ADVANCED PANDAS FUNCTIONALITY

In [33]:
import pandas as pd
import numpy as np

print ('pandas version', pd.__version__)

# Tokyo Olympics
tokyo_data = pd.read_csv('https://raw.githubusercontent.com/laitanawe/pandasds/main/workshops/data/tokyo_olympics.csv')

#  .rank() .rolling() .cumsum() .shift()

tokyo_data['height_rank'] = tokyo_data['height_cm'].rank()
tokyo_data['height_rank'].sort_values(ascending=False)

for index, row in tokyo_data.iterrows():
  print(index)
  print("Athlete: ", row['person_name'])
  print(row['country'])
  print("\n\n")


0
Athlete:   AALERUD Katrine
Norway



1
Athlete:  ABAD Nestor
Spain



2
Athlete:  ABAGNALE Giovanni
Italy



3
Athlete:  ABALDE Alberto
Spain



4
Athlete:  ABALDE Tamara
Spain



5
Athlete:  ABALO Luc
France



6
Athlete:  ABAROA Cesar
Chile



7
Athlete:  ABASS Abobakr
Sudan



8
Athlete:  ABBASALI Hamideh
Islamic Republic



9
Athlete:  ABBASOV Islam
Azerbaijan



10
Athlete:  ABBINGH Lois
Netherlands



11
Athlete:  ABBOT Emily
Australia



12
Athlete:  ABBOTT Monica
United States of America



13
Athlete:  ABDALLA Abubaker
Qatar



14
Athlete:  ABDALLA Maryam
Egypt



15
Athlete:  ABDALLAH Shahd
Egypt



16
Athlete:  ABDALRASOOL Mohamed
Sudan



17
Athlete:  ABDEL LATIF Radwa
Egypt



18
Athlete:  ABDEL RAZEK Samy
Egypt



19
Athlete:  ABDELAZIZ Abdalla
Egypt



